# LLM-as-a-Judge と Iterative DPO によるプリファレンスデータ合成

Susumu Ota  2025-02-02

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/susumuota/synthetic-data-hands-on/blob/main/notebooks/synth_llm_judge.ipynb)

本ハンズオンでは、言語モデルを使ってプリファレンスデータ(preference data, 選好データ)を合成する方法を紹介します。

まず、簡単に [Direct Preference Optimization (DPO)](https://papers.nips.cc/paper_files/paper/2023/hash/a85b405ed65c6477a4fe8302b5e06ce7-Abstract-Conference.html) と [Iterative Direct Preference Learning (Iterative DPO)](https://arxiv.org/abs/2405.07863) について説明します。次に、[LLM-as-a-Judge](https://proceedings.neurips.cc/paper_files/paper/2023/hash/91f18a1287b398d378ef22505bf41832-Abstract-Datasets_and_Benchmarks.html) 手法の解説、そしてそれらの手法を使って実際にプリファレンスデータを合成する手順を紹介します。作成したプリファレンスデータは DPO などの事後学習に利用することができます。

## Direct Preference Optimization (DPO) とは

[Direct Preference Optimization (DPO)](https://papers.nips.cc/paper_files/paper/2023/hash/a85b405ed65c6477a4fe8302b5e06ce7-Abstract-Conference.html) は人間のプリファレンス(選好)に基づいて言語モデルを最適化する手法で、概要は以下の通りです。

<img src="https://github.com/user-attachments/assets/bba0a5d1-b6ba-4c21-aaf7-697169b85db8" width="800"><br />

- 背景
  - 言語モデルの振る舞いを正確に制御することが難しい
  - 既存研究では、振る舞いを制御するために人間のフィードバックを使って強化学習を行う (Reinforcement Learning from Human Feedback, RLHF)
  - しかし RLHF は複雑・不安定・高コスト
- 目的
  - 明示的な報酬モデリングや強化学習を用いずに、人間のプリファレンスに従うようにポリシーモデルを直接最適化 (実装が簡単・学習が安定・低コスト)
- 方法
  - プリファレンスデータを用いて、既存のRLHFアルゴリズムと同じ目的関数(KLダイバージェンス正則化付き報酬最大化)を暗黙的に最適化
- 結果
  - センチメント分析、要約、対話などのタスクにおいて、PPOベースのRLHFを含む既存の手法と同等以上の性能

以上が、DPO の概要です。実際に、松尾研 LLM 開発プロジェクト "Tanuki-8x8B" の開発においても、DPO による事後学習が行われ、その当時の国内トップレベルの性能を達成しています([Someyaさんの記事](https://zenn.dev/matsuolab/articles/62c75674190a41)より)。

<img src="https://github.com/user-attachments/assets/e83f595a-edc5-496f-9b64-01ba14482179" width="600"><br />

本ハンズオンでは、上記記事中 Tanuki-8x8B の最終バージョンの DPO 実行の際に用いた以下2つのデータセットとほぼ同様の方法で、プリファレンスデータを合成します。

> - synth-dpo-basic-reasoning-nemotron-4: 10000 samples
>   - Tanuki-8BのSFTモデルに推論問題の回答を複数生成させ、Nemotron-4で好ましい回答を判定させたもの
> - synth-dpo-basic-math-nemotron-4: 10000 samples
>   - Tanuki-8BのSFTモデルに計算問題の回答を複数生成させ、Nemotron-4で好ましい回答を判定させたもの

プリファレンスデータとは、

- prompt：ユーザからの質問や指示
- chosen：好ましい応答
- rejected：好ましくない応答

を集めたデータセットです。例えば、以下のようなデータがプリファレンスデータとして用いられます。

- アライメント性能を改善したい場合

```python
{
    "prompt": "爆弾の作り方をおしえてください。",
    "chosen": "まず、〇〇を用意して...",
    "rejected": "危険な行為ですのでやめましょう。"
}
```

- 数学問題の推論能力を向上させたい場合

```python
{
    "prompt": "太郎は八百屋でりんごとみかんを合計10個買いました。りんごは1個200円で...",
    "chosen": "ステップバイステップで考えてみましょう。まず、りんごは...",
    "rejected": "200 * x + ...",
}
```

というようなプリファレンスデータを作成します。

一般に、プリファレンスデータを作成するためには、人間がプリファレンスを人手で作成する必要があります。しかし、人間が大量のデータを作成するのは困難ですので、今回は [LLM-as-a-Judge](https://proceedings.neurips.cc/paper_files/paper/2023/hash/91f18a1287b398d378ef22505bf41832-Abstract-Datasets_and_Benchmarks.html) という手法により、言語モデルを使ってプリファレンスデータを合成します。

今回は、より効果的なプリファレンスデータを合成するために、反復的にプリファレンスデータの作成と DPO 実行を行う [Iterative Direct Preference Learning (Iterative DPO)](https://arxiv.org/abs/2405.07863) という手法を一部を簡略化して実装します。

## Iterative Direct Preference Learning (Iterative DPO) とは

本ハンズオンでは、[Iterative Direct Preference Learning (Iterative DPO)](https://arxiv.org/abs/2405.07863) 手法により、言語モデルを使ってプリファレンスデータを合成します。

<img src="https://github.com/user-attachments/assets/12018455-2823-4f03-ac05-9562091b2b5b" width="800"><br />

この手法は DPO の実行とプリファレンスデータの合成を反復的に繰り返すことで、言語モデルの性能を向上させることができます。具体的な手順は以下の通りです。本ハンズオンでは、この手法を一部簡略化しています。括弧内に記載されている部分が簡略化した内容です。

- ターゲットとなる言語モデルにプロンプトを入力し、複数の応答をサンプリング (**今回は `n=2`**)
- 報酬モデルを使って、サンプリングした応答を評価 (**今回は報酬モデルの代わりに LLM-as-a-Judge を使う**)
- 高評価のものを chosen、低評価のものを rejected としてプリファレンスデデータを合成
- DPO を実行して言語モデルを最適化
- 最初に戻る

このステップを反復的に繰り返すことで、言語モデルの性能を徐々に向上させることが出来ると報告されています。

<img src="https://github.com/user-attachments/assets/64700f0c-c56f-40f3-a39e-4852c751347b" width="600"><br />

なお、このハンズオンでは、上記ステップのうち、プリファレンスデータの合成までを紹介します。DPO の実行方法は、[勉強会の過去回](https://www.youtube.com/watch?v=ezNx6tB8jac)や、[knishimae さんの記事](https://zenn.dev/matsuolab/articles/62d99af24ff89a)を参照してください。

## LLM-as-a-Judge 手法の概要

プリファレンスデータを作成するために、ターゲットとなる言語モデルにプロンプトを入力し、2個の応答をサンプリングで生成します。その後、サンプリングした応答を評価するために、LLM-as-a-Judge ([元論文](https://proceedings.neurips.cc/paper_files/paper/2023/hash/91f18a1287b398d378ef22505bf41832-Abstract-Datasets_and_Benchmarks.html), [サーベイ プレプリント](https://arxiv.org/abs/2411.15594)) 手法を使います。ここでは LLM-as-a-Judge 手法の概要を紹介します。

- 背景
  - 正確で一貫性のある評価は、様々な分野での意思決定において重要であるが、内在的な主観性・ばらつき・スケールが存在するため困難なタスク
- 目的
  - 従来の人手(専門家)の評価に代わり、言語モデルで多様なタイプのデータを処理し、スケーラブルでコスト効率に優れ一貫性のある評価を提供
- 方法
  - 言語モデルを評価者として利用し3種類の判定方法を提案
    - ペアワイズ比較: 質問と2つの応答を提示し、どちらが優れているか、または同点を判定
    - 単一解答の採点: 1つの応答に直接得点を割り当てる
    - リファレンス付き採点: リファレンスとなる応答を提示し、その上でペアワイズ比較や単一解答の採点を行う
- 結果
  - GPT-4 による判定は人間の評価と85%の割合で一致し、人間-人間の結果81%と同程度の一致率を達成
  - 位置バイアス(position bias)、冗長性バイアス(verbosity bias)、自己強化バイアス(self-enhancement bias)、数学や推論能力の限界などの問題点

以下の図は、ペアワイズ比較のプロンプト例で、今回はこちらのプロンプトを使ってプリファレンスデータを合成します。

<img src="https://github.com/user-attachments/assets/12111a26-50c8-439c-a23a-ae7240c259bc" width="800"><br />

以下の図は、6つの言語モデル(ターゲット言語モデル)を対象として、様々な3つの裁判官言語モデルと人手で判定した結果です。人間と比べると、特定のモデルを好む裁判官言語モデルがあることが明らかになりました。例えば、GPT-4は自分の勝率を10%高く、Claude-v1は自分の勝率を25%高く評価する傾向があります(自己強化バイアス, self-enhancement bias の可能性)。

<img src="https://github.com/user-attachments/assets/853e351d-f35b-41f3-be28-29426feede89" width="800"><br />

以下の表は、GPT-4 による判定は人間の評価と85%の割合で一致し、人間-人間の結果81%と同程度の一致率を達成したことを示しています。

<img src="https://github.com/user-attachments/assets/12c28e36-cce5-4361-b32f-72ccae8ef5f4" width="800"><br />

なお、位置バイアス(position bias)については、後ほど紹介します。

## 準備

準備については、[Persona-Hub による事後学習データ合成](https://github.com/susumuota/synthetic-data-hands-on/blob/main/notebooks/synth_persona.ipynb)ハンズオンの内容とほぼ同じですのでそちらの資料を参照してください。

### トークンと API キーの設定

In [1]:
if str(get_ipython()).startswith("<google.colab"):  # if this notebook is running in Google Colab  # type: ignore
    import os
    from google.colab import userdata  # type: ignore

    os.environ["HF_TOKEN"] = userdata.get("HF_TOKEN")
    os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")
    # os.environ["NVIDIA_NIM_API_KEY"] = userdata.get("NVIDIA_NIM_API_KEY")

### モジュールのインストール

In [2]:
# %pip install litellm
# %pip install datasets
# %pip install pandas

# %pip install vllm

### モジュールのインポート

In [3]:
from abc import ABC, abstractmethod
import json
from logging import DEBUG, INFO, StreamHandler, getLogger  # noqa: F401
import pprint


from datasets import load_dataset
from litellm import batch_completion
import pandas as pd


try:
    from vllm import LLM, SamplingParams  # type: ignore
except ImportError:
    print("No vllm module found. You can only use the LiteLLM.")

/Users/ota/Documents/python/synthetic-data-hands-on/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/ota/Documents/python/synthetic-data-hands-on/.venv/lib/python3.11/site-packages/pydantic/_internal/_config.py:345: UserWarning: Valid config keys have changed in V2:
* 'fields' has been removed
  warnings.warn(message, UserWarning)


No vllm module found. You can only use the LiteLLM.


### ログの設定

In [4]:
logging_level = DEBUG
# logging_level = INFO  # uncomment if you want to see less output
logger = getLogger(__name__)
logger.setLevel(logging_level)
handler = StreamHandler()
handler.setLevel(logging_level)
logger.addHandler(handler)

pp = pprint.PrettyPrinter(width=100)

### 言語モデルの設定

`n` というパラメータを追加して、一つのプロンプトに対して複数の結果を生成するよう変更しました。結果出力もリストのリストに変更となります。 `# changed` というコメントがある行が変更された行です。

In [5]:
class LanguageModel(ABC):
    def __init__(self, model: str, temperature=1.0, max_tokens=16, seed=None, n=None):  # changed
        self.model = model
        self.temperature = temperature
        self.max_tokens = max_tokens
        self.seed = seed
        self.n = n  # changed
        logger.debug(f"model: {model}, temperature: {temperature}, max_tokens: {max_tokens}, seed: {seed}, n: {n}")  # changed

    @abstractmethod
    def __call__(self, messages_batch: list[list[dict[str, str]]]) -> list[list[str]]:  # changed
        pass


class LiteLLMModel(LanguageModel):
    def __init__(self, model: str, temperature=1.0, max_tokens=16, seed=None, n=None):  # changed
        super().__init__(model, temperature, max_tokens, seed, n)  # changed

    def __call__(self, messages_batch: list[list[dict[str, str]]]) -> list[list[str]]:  # changed
        contents = [
            [choice.message.content for choice in response.choices]  # changed
            for response in batch_completion(
                model=self.model,
                messages=messages_batch,
                temperature=self.temperature,
                max_tokens=self.max_tokens,
                seed=self.seed,
                n=self.n,  # changed
            )
        ]
        assert len(contents) == len(messages_batch)
        return contents


class VLLMModel(LanguageModel):
    def __init__(self, model: str, temperature=1.0, max_tokens=16, seed=None, n=None, dtype="auto", stop=None):  # changed
        super().__init__(model, temperature, max_tokens, seed, n)  # changed
        self.dtype = dtype
        self.stop = stop
        self.vllm = LLM(model, dtype=dtype)  # dtype must be "half" to run on Colab T4
        self.tokenizer = self.vllm.get_tokenizer()

    def __call__(self, messages_batch: list[list[dict[str, str]]]) -> list[list[str]]:  # changed
        sampling_params = SamplingParams(
            temperature=self.temperature, max_tokens=self.max_tokens, seed=self.seed, n=self.n, stop=self.stop  # changed
        )
        prompts = [
            self.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
            for messages in messages_batch
        ]
        outputs = self.vllm.generate(prompts, sampling_params=sampling_params, use_tqdm=False)
        contents = [
            [oo.text for oo in o.outputs]  # changed
            for o in outputs
        ]
        assert len(contents) == len(messages_batch)
        return contents

### 言語モデルの作成

裁判官の役割をする言語モデルを作成します。温度パラメータ `temperature` が `0.0` であることに注意してください。`0.0` の場合はサンプリングせずにグリーディに最も確率の高いトークンを選択します。また、`n` は指定しませんがデフォルト値は `1` です。

Note: 裁判官のタスクは難易度が高いので、高度な言語モデルを使用する必要があります。10B 程度の言語モデルでは難しいかもしれません(タスクを理解できない可能性があります)。

Note: Colab T4 で実行する場合、GPU メモリがギリギリですので、`judge_llm` は API を使い、`target_llm` はローカル GPU を使うように設定してください。

In [6]:
judge_llm = LiteLLMModel("gpt-4o-mini", temperature=0.0, max_tokens=512, seed=0)  # OPENAI_API_KEY
# judge_llm = LiteLLMModel("nvidia_nim/nvidia/nemotron-4-340b-instruct", temperature=0.0, max_tokens=512, seed=None)  # NVIDIA_NIM_API_KEY
# judge_llm = VLLMModel("team-hatakeyama-phase2/Tanuki-8B-dpo-v1.0-GPTQ-8bit", temperature=0.0, max_tokens=512, seed=0)  # for Colab T4
# judge_llm = VLLMModel("marcsun13/gemma-2-9b-it-GPTQ", temperature=0.0, max_tokens=512, seed=0)  # for Colab T4
# judge_llm = VLLMModel("cyberagent/calm3-22b-chat", temperature=0.0, max_tokens=512, seed=0)  # for A100?

judge_llm

model: gpt-4o-mini, temperature: 0.0, max_tokens: 512, seed: 0, n: None


In [7]:
judge_llm([[{"role": "user", "content": "Hello?"}]])

[['Hello! How can I assist you today?']]

## プロンプトの読み込み

プロンプトは事前に用意したものを利用します。以下 URL からダウンロードします。

In [8]:
!wget https://raw.githubusercontent.com/susumuota/synthetic-data-hands-on/refs/heads/main/notebooks/basic_math_mt.jsonl

--2025-02-01 23:27:23--  https://raw.githubusercontent.com/susumuota/synthetic-data-hands-on/refs/heads/main/notebooks/basic_math_mt.jsonl
raw.githubusercontent.com (raw.githubusercontent.com) をDNSに問いあわせています... 185.199.109.133, 185.199.108.133, 185.199.111.133, ...
raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443 に接続しています... 接続しました。
HTTP による接続要求を送信しました、応答を待っています... 200 OK
長さ: 12635 (12K) [text/plain]
`basic_math_mt.jsonl.1' に保存中

basic_math_mt.jsonl 100%[===================>]  12.34K  --.-KB/s 時間 0s         

2025-02-01 23:27:24 (34.0 MB/s) - `basic_math_mt.jsonl.1' へ保存完了 [12635/12635]



ここでは、Persona-Hub のハンズオンで作成したマルチターンの対話データから最初の質問 `Q1` だけを取り出しプロンプトとします。

In [9]:
def load_questions(input_jsonl: str) -> list[str]:
    dataset = load_dataset("json", data_files=input_jsonl, name="default", split="train", cache_dir="cache")
    batch = pd.DataFrame(dataset).to_dict(orient="records")  # convert column-wise to row-wise
    return [b["messages"][0]["content"] for b in batch]

In [10]:
questions = load_questions("basic_math_mt.jsonl")
questions

['ニュー・ジャージー・パインバレンズの生態系を研究している植物研究者がいます。彼は、特定の植物のサンプルを集めるために、5つの異なる地点を訪れました。各地点で、彼は次のように植物を収集しました：地点Aで12本、地点Bで15本、地点Cで8本、地点Dで10本、地点Eで9本です。彼が集めた植物の総数はいくつですか？',
 'ある博物館で、古代の節足動物の化石が展示されています。展示されている化石は、クモが10体、サソリが15体、エビが20体です。博物館の見学者がそれぞれの種類の化石を見学した後、クモの化石を見た見学者の数がサソリの化石を見た見学者の数よりも2倍多いとします。クモの化石を見た見学者が20人だった場合、サソリの化石を見た見学者は何人ですか？',
 'あるプログラミング言語の研究者が、数値計算のアルゴリズムをテストするために、10個の異なる入力データセットを用意しました。彼は各データセットに対して3つの異なるアルゴリズムを試すことにしました。全てのデータセットとアルゴリズムの組み合わせを考えると、彼は合計で何回テストを実施することになりますか？',
 'ある流体力学のエンジニアは、水槽の自由表面の面積を計算しています。水槽の形は長方形で、長さが6メートル、幅が4メートルです。この水槽が満水のとき、水面の面積は何平方メートルですか？',
 'ある年、サーモンが川を遡上するために、1日あたり30キロメートル進みました。サーモンが川を上り始めてから、合計で6日間進みました。サーモンは合計で何キロメートル上ったでしょうか？',
 'ある宇宙ロボット工学者が、火星探査のために新しいロボットを設計しています。このロボットは、1時間で10キロメートルの速度で移動します。もしロボットが火星の表面を30時間移動し続けた場合、ロボットは合計で何キロメートル移動することになりますか？',
 '環境ジャーナリストであるあなたは、オーガニック農法で作物を育てる農家を取材しています。農家は、1エーカーの土地で年間に2000ポンドのオーガニック野菜を収穫します。また、1エーカーの土地に蜂の巣箱を5つ設置すると、収穫量が10％増加します。あなたは、蜂の巣箱を3つ設置した場合の収穫量を計算したいと思っています。蜂の巣箱を3つ設置したときの収穫量は何ポンドになりますか？',
 'ある小学校の先生

## サンプリングで 1個の質問から2個の異なる解答を生成

### 温度パラメータの探索

サンプリングで1個の質問に対して2個の異なる解答を生成します。ただし、解答が同じにならないように温度パラメータを調整します。以下のような方針で探索を行います。

- 1個の質問から2個の解答を生成
- 生成された2個の解答を比較
- 2個の解答が異なっていたらOK
- もし一致していたらNG、温度を少し上げて再度解答を生成

現在学習しようとしている言語モデル(`target_llm`)を初期化します(裁判官言語モデルとは別)。必ず `n=2` としてください。

`target_llm` は現在学習をしようとしているモデルですので、必ずしも高度な言語モデルである必要はありません。

In [11]:
target_llm = LiteLLMModel("gpt-4o-mini", temperature=0.7, max_tokens=512, seed=0, n=2)  # OPENAI_API_KEY
# llm = LiteLLMModel("nvidia_nim/nvidia/nemotron-4-340b-instruct", temperature=0.7, max_tokens=512, seed=None, n=2)  # NVIDIA_NIM_API_KEY
# target_llm = VLLMModel("marcsun13/gemma-2-9b-it-GPTQ", temperature=0.7, max_tokens=512, seed=0, n=2)  # for Colab T4
# target_llm = VLLMModel("team-hatakeyama-phase2/Tanuki-8B-dpo-v1.0-GPTQ-8bit", temperature=0.7, max_tokens=512, seed=0, n=2)  # for Colab T4

model: gpt-4o-mini, temperature: 0.7, max_tokens: 512, seed: 0, n: 2


下のセルの `target_llm.temperature = 0.7` の部分を低い温度から始めて、全ての解答が異なるまで温度を徐々に上げて最適な `temperature` を探してください。上げすぎるとハルシネーションを起こしたり、文が破綻するので注意してください。

以下のセルを何度か手動で実行してみて、`differences` の要素がほとんど True になることを確認してください。多少 False が出力されても問題ありませんが、False が多い場合は温度を調整してください。

In [12]:
target_llm.temperature = 0.7
target_llm.n = 2

answers = target_llm([[{"role": "user", "content": question}] for question in questions])

differences = [answer[0] != answer[1] for answer in answers]
all(differences), differences

(True, [True, True, True, True, True, True, True, True, True, True])

ここでは `temperature` を `0.7` 程度に設定していますが、最適な `temperature` は言語モデルによって異なります。

Note: [vLLM の推論オプション](https://docs.vllm.ai/en/stable/api/inference_params.html)で `best_of` というパラメータがあります。これは複数のサンプリング結果から、最もスコアの高い `n` 個の応答を選択するためのオプションです。例えば、`best_of=4`, `n=2` とすると、4個のサンプリング結果から最もスコアの良い2個の応答を選択します(その分推論コストは増えます)。このオプションを使うことで、より効果的な結果を得られる可能性があります。`Tanuki-8x8B` 開発の際は `best_of=4`, `n=2` としてサンプリングし、スコア上位2個の応答を使いましたが、例えば `best_of=4`, `n=4` として、最もスコアの高い応答と最も低い応答をペアにするという方法も考えられます。もし `best_of` を使いたい場合は、`VLLMModel` を修正して `best_of` を指定してください。

### サンプリングで解答を2個生成

以下のような関数で解答を2個生成します。

In [13]:
def generate_answers(llm: LanguageModel, questions: list[str]) -> list[str]:
    llm.n = 2
    answers = llm([[{"role": "user", "content": question}] for question in questions])
    return [
        {"question": question, "answer_a": answer[0], "answer_b": answer[1]}
        for question, answer in zip(questions, answers)
    ]

実行して確認します。

In [14]:
answers = generate_answers(target_llm, questions)
answers

[{'question': 'ニュー・ジャージー・パインバレンズの生態系を研究している植物研究者がいます。彼は、特定の植物のサンプルを集めるために、5つの異なる地点を訪れました。各地点で、彼は次のように植物を収集しました：地点Aで12本、地点Bで15本、地点Cで8本、地点Dで10本、地点Eで9本です。彼が集めた植物の総数はいくつですか？',
  'answer_a': '植物研究者が集めた植物の総数を計算するために、各地点での植物の本数を足します。\n\n地点A: 12本  \n地点B: 15本  \n地点C: 8本  \n地点D: 10本  \n地点E: 9本  \n\nこれらを合計すると、\n\n12 + 15 + 8 + 10 + 9 = 54\n\nしたがって、彼が集めた植物の総数は54本です。',
  'answer_b': '植物研究者が集めた植物の総数を求めるために、各地点で収集した植物の本数を足し合わせます。\n\n地点A: 12本  \n地点B: 15本  \n地点C: 8本  \n地点D: 10本  \n地点E: 9本  \n\nこれらを合計すると、\n\n12 + 15 + 8 + 10 + 9 = 54\n\nしたがって、彼が集めた植物の総数は54本です。'},
 {'question': 'ある博物館で、古代の節足動物の化石が展示されています。展示されている化石は、クモが10体、サソリが15体、エビが20体です。博物館の見学者がそれぞれの種類の化石を見学した後、クモの化石を見た見学者の数がサソリの化石を見た見学者の数よりも2倍多いとします。クモの化石を見た見学者が20人だった場合、サソリの化石を見た見学者は何人ですか？',
  'answer_a': 'クモの化石を見た見学者の数を \\( C \\)、サソリの化石を見た見学者の数を \\( S \\) とします。\n\n問題文から、クモの化石を見た見学者の数 \\( C \\) はサソリの化石を見た見学者の数 \\( S \\) よりも2倍多いということがわかります。この関係を式で表すと次のようになります：\n\n\\[\nC = 2S\n\\]\n\nまた、クモの化石を見た見学者の数は20人と与えられているので、これを式に代入します：\n\n\\[\n20 = 2S\n\\]\n\nこの

2個の解答が異なるかどうかチェックします。

In [15]:
def check_answers(answers: list[dict[str, str]]) -> list[bool]:
    return [a["answer_a"] != a["answer_b"] for a in answers]

In [16]:
results = check_answers(answers)
all(results), results

(True, [True, True, True, True, True, True, True, True, True, True])

もし同じ解答が多い(False の数が多い)場合は、言語モデルの設定で温度パラメータを上げて調整してください。

## LLM-as-a-Judge でどちらの解答が優れているか判定

ここからは、生成された2個の解答を LLM-as-a-Judge 手法でどちらの解答が優れているか判決を下します。

今回は [LLM-as-a-Judge 論文](https://proceedings.neurips.cc/paper_files/paper/2023/hash/91f18a1287b398d378ef22505bf41832-Abstract-Datasets_and_Benchmarks.html)の [GitHub リポジトリにあるプロンプト](https://github.com/lm-sys/FastChat/blob/main/fastchat/llm_judge/data/judge_prompts.jsonl)の `pair-v2` というバージョンを使用して、2つの解答を比較します。

<img src="https://github.com/user-attachments/assets/12111a26-50c8-439c-a23a-ae7240c259bc" width="800">

In [17]:
# copy from https://github.com/lm-sys/FastChat/blob/main/fastchat/llm_judge/data/judge_prompts.jsonl

JUDGE_PROMPT = {
    "name": "pair-v2",
    "type": "pairwise",
    "system_prompt": "Please act as an impartial judge and evaluate the quality of the responses provided by two AI assistants to the user question displayed below. You should choose the assistant that follows the user's instructions and answers the user's question better. Your evaluation should consider factors such as the helpfulness, relevance, accuracy, depth, creativity, and level of detail of their responses. Begin your evaluation by comparing the two responses and provide a short explanation. Avoid any position biases and ensure that the order in which the responses were presented does not influence your decision. Do not allow the length of the responses to influence your evaluation. Do not favor certain names of the assistants. Be as objective as possible. After providing your explanation, output your final verdict by strictly following this format: \"[[A]]\" if assistant A is better, \"[[B]]\" if assistant B is better, and \"[[C]]\" for a tie.",
    "prompt_template": "[User Question]\n{question}\n\n[The Start of Assistant A's Answer]\n{answer_a}\n[The End of Assistant A's Answer]\n\n[The Start of Assistant B's Answer]\n{answer_b}\n[The End of Assistant B's Answer]",
    "description": "Prompt for general questions",
    "category": "general",
    "output_format": "[[A]]"
}

上記の `system_prompt` と `prompt_template` を使用します。

システムプロンプトの日本語訳は以下の通りです。

> 公平な裁判官として、以下に表示されたユーザーの質問に対して2人のAIアシスタントが提供した回答の質を評価してください。あなたは、ユーザーの指示に従い、ユーザーの質問によりよく答えるアシスタントを選ぶべきです。あなたの評価は、回答の有用性、関連性、正確性、深さ、創造性、詳細レベルなどの要素を考慮する必要があります。2つの回答を比較し、簡単な説明をすることから評価を始めてください。立場が偏らないようにし、回答の提示順があなたの判断に影響しないようにしてください。回答の長さが評価に影響しないようにしてください。特定のアシスタントの名前を好まないこと。できるだけ客観的であること。説明の後、以下の書式にしたがって最終評価を出力してください： アシスタントAが優れていれば「[[A]]」、アシスタントBが優れていれば「[[B]]」、同点の場合は「[[C]]」とします。

判決文を生成する関数を定義します。

In [18]:
def generate_decision_texts(llm: LanguageModel, answers: list[dict[str, str]]) -> list[str]:
    return [texts[0] for texts in llm([[
        {"role": "system", "content": JUDGE_PROMPT["system_prompt"]},
        {"role": "user", "content": JUDGE_PROMPT["prompt_template"].format(**answer)}
    ] for answer in answers])]

実際に判定をしてみます。判決文(`decision_texts`)の末尾に `[[A]]`, `[[B]]`, `[[C]]` のいずれかが含まれているかを確認してください。

Note: このタスクは難易度が高いので、`judge_llm` は高度な言語モデルを使用する必要があります。

In [19]:
decision_texts = generate_decision_texts(judge_llm, answers)
decision_texts

["Both Assistant A and Assistant B provided identical responses to the user's question, accurately calculating the total number of plants collected by the researcher. They both listed the number of plants collected at each location, performed the addition correctly, and arrived at the same conclusion of 54 plants.\n\nIn terms of helpfulness, relevance, accuracy, depth, creativity, and level of detail, both responses are equal. They both clearly outline the steps taken to arrive at the answer and provide the correct total.\n\nSince there is no discernible difference in the quality of the responses, the evaluation leads to a tie.\n\nFinal verdict: [[C]]",
 "Assistant A provides a clear and accurate solution to the problem. It correctly identifies the relationship between the number of visitors who saw the spider fossils and the scorpion fossils, sets up the equation based on the given information, and solves it step by step to arrive at the correct answer of 10 visitors for the scorpion 

解答と判決文を表示して中身を確認します。

In [20]:
pp.pprint(list(zip(answers, decision_texts)))

[({'answer_a': '植物研究者が集めた植物の総数を計算するために、各地点での植物の本数を足します。\n'
               '\n'
               '地点A: 12本  \n'
               '地点B: 15本  \n'
               '地点C: 8本  \n'
               '地点D: 10本  \n'
               '地点E: 9本  \n'
               '\n'
               'これらを合計すると、\n'
               '\n'
               '12 + 15 + 8 + 10 + 9 = 54\n'
               '\n'
               'したがって、彼が集めた植物の総数は54本です。',
   'answer_b': '植物研究者が集めた植物の総数を求めるために、各地点で収集した植物の本数を足し合わせます。\n'
               '\n'
               '地点A: 12本  \n'
               '地点B: 15本  \n'
               '地点C: 8本  \n'
               '地点D: 10本  \n'
               '地点E: 9本  \n'
               '\n'
               'これらを合計すると、\n'
               '\n'
               '12 + 15 + 8 + 10 + 9 = 54\n'
               '\n'
               'したがって、彼が集めた植物の総数は54本です。',
   'question': 'ニュー・ジャージー・パインバレンズの生態系を研究している植物研究者がいます。彼は、特定の植物のサンプルを集めるために、5つの異なる地点を訪れました。各地点で、彼は次のように植物を収集しました：地点Aで12本、地点Bで15本、地点Cで8本、地点Dで10本、地点Eで9本です。彼が集めた植物の総数はいくつですか？'},
  "Both Assis

判決文をパースして、`A`, `B`, `C` のいずれかを返す関数を定義します。明確に決められないケースは `C` とします。

In [21]:
def parse_decision_text(decision_text):
    is_a = is_b = is_c = False
    if "[[A]]" in decision_text:
        is_a = True
    if "[[B]]" in decision_text:
        is_b = True
    if "[[C]]" in decision_text:
        is_c = True
    decision = "C"
    if is_a and is_b:
        # raise ValueError(f"Both A and B are chosen: {decision_text}")
        logger.debug(f"Both A and B are chosen: {decision_text}")
        decision = "C"
    elif is_a:
        decision = "A"
    elif is_b:
        decision = "B"
    elif is_c:
        decision = "C"
    else:
        # raise ValueError(f"Unknown decision: {decision_text}")
        logger.debug(f"Unknown decision: {decision_text}")
        decision = "C"
    # logger.debug(f"decision: {decision}")
    return decision

解答をジャッジして判決を返す関数にまとめます。

In [22]:
def judge_answers(llm: LanguageModel, answers: list[dict[str, str]]) -> list[str]:
    return [parse_decision_text(decision_text) for decision_text in generate_decision_texts(llm, answers)]

実行して、解答に対する判定を確認します。

In [23]:
decisions = judge_answers(judge_llm, answers)
decisions

['C', 'A', 'B', 'A', 'A', 'C', 'B', 'B', 'C', 'B']

ここまでで、1個の質問に対して2個の解答を生成し、それらの解答を LLM-as-a-Judge で判定することができました。

この判定結果から、引き分け `C` 判定を取り除いて `A` と `B` の判決をした解答のみからプリファレンスデータを生成することになりますが、その前に位置バイアスの影響を取り除く処理を行います。

## 位置バイアスの軽減

位置バイアス (position bias) とは、言語モデルがある特定の位置を他の位置よりも好む傾向を示すことです。このバイアスは言語モデル特有というわけではなく、人間の意思決定や他の機械学習領域でも見られます([LLM-as-a-Judgeの論文](https://arxiv.org/abs/2306.05685)より)。

例えば、最初に提示された解答が後に提示された解答よりも高い評価を受ける傾向がある、という位置バイアスが存在することが知られています。

<img src="https://github.com/user-attachments/assets/3f3d8c4b-0efe-47a7-8d63-1cd8f672e50c" width="600"><br />

以下のテーブルの `Consistency` が位置バイアスを受けずに一貫した判決を下すことの出来る割合です。最も良い結果でも 65% 前後ですので、位置バイアスの影響が大きいことがわかります。また、全体としては、最初の選択肢を選ぶバイアスが大きいこともわかります。

<img src="https://github.com/user-attachments/assets/4b4c4441-8ea6-4e06-9154-a69d49767eed" width="600"><br />

位置バイアスの影響を減らすための簡単な解決策として、解答の順番をスワップして再度ジャッジして、判決に一貫性がある場合のみ勝利とするという方法があります。スワップした後で結果が一致しない場合は、引き分けとします。

今回はこの方法を使って位置バイアスの影響を軽減します。

まず、解答の順番を入れ替える関数を定義します。

In [24]:
def swap_answers(answers: list[dict[str, str]]) -> list[dict[str, str]]:
    return [{"question": a["question"], "answer_a": a["answer_b"], "answer_b": a["answer_a"]} for a in answers]

スワップしてから判定した結果は A と B の順番が変わっていますので、比較する際は A と B を入れ替えて比較します。判決を入れ替える関数を定義します。

In [25]:
def swap_decisions(decisions: list[str]) -> list[str]:
    return ["A" if d == "B" else "B" if d == "A" else "C" for d in decisions]

実際にスワップして判定してみます。

In [26]:
swapped_decisions = swap_decisions(judge_answers(judge_llm, swap_answers(answers)))
swapped_decisions

['C', 'A', 'B', 'A', 'C', 'C', 'B', 'A', 'B', 'B']

元の判定とスワップした判定を比較して、判決が一致していて、かつ、`C` が含まれていない場合をプリファレンスデータとして採用します。

In [27]:
def verify_decisions(decisions: list[str], swapped_decisions: list[str]) -> list[list[str]]:
    return [d1 == d2 and d1 != "C" for d1, d2 in zip(decisions, swapped_decisions)]

In [28]:
verifications = verify_decisions(decisions, swapped_decisions)
list(zip(verifications, decisions, swapped_decisions))

[(False, 'C', 'C'),
 (True, 'A', 'A'),
 (True, 'B', 'B'),
 (True, 'A', 'A'),
 (False, 'A', 'C'),
 (False, 'C', 'C'),
 (True, 'B', 'B'),
 (False, 'B', 'A'),
 (False, 'C', 'B'),
 (True, 'B', 'B')]

True が位置を入れ替えても判決がゆるがない(バイアスを受けていない)という結果です、False が位置を入れ替えると判決が覆ってしまう(位置バイアス)場合と引き分けの場合です。

位置をスワップしても判決が変わらなかったのは約半数ということになりました。

ここでの検討事項は、

- 判決が入れ替わった数がどの程度あるか (バイアスあるいはタスクの理解ができていない)
- A (あるいは B)を選ぶ傾向が明確にあるか (バイアスがあるかどうか)
- C を含むケースをプリファレンスデータから除外するかどうか (例えば `(False, B, C)` のようなケースをBの勝ちとするか、あるいは除外するか)

となります。

今回は、引き分け C を含むケースを全て除外した上で、バイアスを受けなかったデータをプリファレンスデータとして採用します (`verified` が `True` のデータ)。今までの処理を関数にまとめます。

In [29]:
def generate_preferences(judge_llm: LanguageModel, target_llm: LanguageModel, questions: list[str]) -> list[dict[str, str | bool]]:
    answers = generate_answers(target_llm, questions)
    logger.debug(f"len(answers): {len(answers)}")
    decisions = judge_answers(judge_llm, answers)
    logger.debug(f"decisions: {decisions}")
    swapped_decisions = swap_decisions(judge_answers(judge_llm, swap_answers(answers)))
    logger.debug(f"swapped_decisions: {swapped_decisions}")
    verifications = verify_decisions(decisions, swapped_decisions)
    logger.debug(f"verifications: {verifications}")
    return [
        {
            "prompt": answer["question"],
            "chosen": answer["answer_a"] if decision == "A" else answer["answer_b"],
            "rejected": answer["answer_b"] if decision == "A" else answer["answer_a"],
            "decision": decision,
            "swapped_decision": swapped_decision,
            "verification": verification,
        }
        for answer, decision, swapped_decision, verification in zip(answers, decisions, swapped_decisions, verifications)
    ]

In [30]:
preferences = generate_preferences(judge_llm, target_llm, questions)
preferences

len(answers): 10
decisions: ['C', 'A', 'C', 'A', 'A', 'C', 'C', 'B', 'C', 'C']
swapped_decisions: ['C', 'A', 'C', 'A', 'C', 'B', 'B', 'A', 'B', 'B']
verifications: [False, True, False, True, False, False, False, False, False, False]


[{'prompt': 'ニュー・ジャージー・パインバレンズの生態系を研究している植物研究者がいます。彼は、特定の植物のサンプルを集めるために、5つの異なる地点を訪れました。各地点で、彼は次のように植物を収集しました：地点Aで12本、地点Bで15本、地点Cで8本、地点Dで10本、地点Eで9本です。彼が集めた植物の総数はいくつですか？',
  'chosen': '植物研究者が集めた植物の総数を求めるために、各地点で収集した植物の本数を足し合わせます。\n\n地点A: 12本  \n地点B: 15本  \n地点C: 8本  \n地点D: 10本  \n地点E: 9本  \n\nこれらを合計すると、\n\n12 + 15 + 8 + 10 + 9 = 54\n\nしたがって、彼が集めた植物の総数は54本です。',
  'rejected': '植物研究者が集めた植物の総数を計算するために、各地点での植物の本数を足します。\n\n地点A: 12本  \n地点B: 15本  \n地点C: 8本  \n地点D: 10本  \n地点E: 9本  \n\nこれらを合計すると、\n\n12 + 15 + 8 + 10 + 9 = 54\n\nしたがって、彼が集めた植物の総数は54本です。',
  'decision': 'C',
  'swapped_decision': 'C',
  'verification': False},
 {'prompt': 'ある博物館で、古代の節足動物の化石が展示されています。展示されている化石は、クモが10体、サソリが15体、エビが20体です。博物館の見学者がそれぞれの種類の化石を見学した後、クモの化石を見た見学者の数がサソリの化石を見た見学者の数よりも2倍多いとします。クモの化石を見た見学者が20人だった場合、サソリの化石を見た見学者は何人ですか？',
  'chosen': 'クモの化石を見た見学者の数が20人であるとします。問題によれば、クモの化石を見た見学者の数はサソリの化石を見た見学者の数の2倍多いとされています。\n\nこれを数式で表すと、サソリの化石を見た見学者の数を \\( x \\) とすると、以下の関係が成り立ちます。\n\n\\[\n20 = 2x\n\\]\n\nこの方程式を解くと、両辺を2で割ります。\n\n\\[\n

jsonl 形式でデータを読み込み、結果を jsonl 形式で保存する関数を作成します。除外したデータは `skip_jsonl` ファイルに保存します。

In [31]:
def run_generate_preferences(judge_llm: LanguageModel, target_llm: LanguageModel, input_jsonl: str = "input.jsonl", output_jsonl: str = "output.jsonl", skip_jsonl: str = "skip.jsonl") -> None:
    questions = load_questions(input_jsonl)
    logger.debug(f"len(questions): {len(questions)}")
    preferences = generate_preferences(judge_llm, target_llm, questions)
    logger.debug(f"len(preferences): {len(preferences)}")
    with open(output_jsonl, "a", encoding="utf-8") as f_output, open(skip_jsonl, "a", encoding="utf-8") as f_skip:
        for preference in preferences:
            f = f_output if preference["verification"] else f_skip
            f.write(json.dumps(preference, ensure_ascii=False) + "\n")
            print(pp.pformat(preference)) if preference["verification"] else None

In [32]:
run_generate_preferences(judge_llm, target_llm, "basic_math_mt.jsonl", "verified_preference.jsonl", "skipped_preference.jsonl")

len(questions): 10
len(answers): 10
decisions: ['C', 'B', 'B', 'A', 'A', 'C', 'A', 'A', 'A', 'B']
swapped_decisions: ['C', 'B', 'C', 'A', 'A', 'B', 'B', 'B', 'A', 'B']
verifications: [False, True, False, True, True, False, False, False, True, True]
len(preferences): 10


{'chosen': 'クモの化石を見た見学者の数が20人で、これがサソリの化石を見た見学者の数よりも2倍多いとします。\n'
           '\n'
           'この情報を数式にすると、次のようになります。\n'
           '\n'
           '- クモの化石を見た見学者の数 = 20人\n'
           '- サソリの化石を見た見学者の数 = \\( x \\) 人\n'
           '\n'
           '与えられた条件より、\n'
           '\\[ 20 = 2x \\]\n'
           '\n'
           'この方程式を解くと、次のようになります。\n'
           '\n'
           '1. 両辺を2で割ります。\n'
           '\\[ x = \\frac{20}{2} \\]\n'
           '\\[ x = 10 \\]\n'
           '\n'
           'したがって、サソリの化石を見た見学者の数は **10人** です。',
 'decision': 'B',
 'prompt': 'ある博物館で、古代の節足動物の化石が展示されています。展示されている化石は、クモが10体、サソリが15体、エビが20体です。博物館の見学者がそれぞれの種類の化石を見学した後、クモの化石を見た見学者の数がサソリの化石を見た見学者の数よりも2倍多いとします。クモの化石を見た見学者が20人だった場合、サソリの化石を見た見学者は何人ですか？',
 'rejected': 'クモの化石を見た見学者の数が20人であるとします。問題文によれば、クモの化石を見た見学者の数はサソリの化石を見た見学者の数よりも2倍多いとされています。これを数式で表すと以下のようになります。\n'
             '\n'
             'クモの化石を見た見学者の数 = 2 × サソリの化石を見た見学者の数\n'
             '\n'
             'したがって、\n'
             '\n'
             '20 = 2 × サソリの化石を見た見学

位置バイアスを取り除いたプリファレンスデータが `verified_preference.jsonl` に保存され、除外したデータが `skipped_preference.jsonl` に保存されます。

この後、`verified_preference.jsonl` を使って、DPO を実行し、言語モデルの性能を向上させることができます。

さらに、DPO で学習した言語モデルを使って、再度新たなプリファレンスデータを合成して DPO を行うことで、反復的に性能を向上させることが可能と報告されています ([Iterative Direct Preference Learning (Iterative DPO)](https://arxiv.org/abs/2405.07863)より)。

<img src="https://github.com/user-attachments/assets/12018455-2823-4f03-ac05-9562091b2b5b" width="800"><br />

本ハンズオンは上記の図のうち、以下のステップを実行したことになります。

- ターゲットとなる言語モデルにプロンプトを入力し、2個の応答をサンプリング
- LLM-as-a-Judge でサンプリングした応答を評価 (+ 位置バイアスの軽減)
- 高評価のものを chosen、低評価のものを rejected としてプリファレンスデータを合成

## まとめ


本ハンズオンでは、言語モデルを使ってプリファレンスデータ(preference data, 選好データ)を合成する方法を紹介しました。以下、本ハンズオンで紹介した内容をまとめます。

- [Direct Preference Optimization (DPO)](https://papers.nips.cc/paper_files/paper/2023/hash/a85b405ed65c6477a4fe8302b5e06ce7-Abstract-Conference.html) と [Iterative Direct Preference Learning (Iterative DPO)](https://arxiv.org/abs/2405.07863) の紹介
- [LLM-as-a-Judge](https://proceedings.neurips.cc/paper_files/paper/2023/hash/91f18a1287b398d378ef22505bf41832-Abstract-Datasets_and_Benchmarks.html) 手法の解説
- [Iterative DPO](https://arxiv.org/abs/2405.07863) 手法によるプリファレンスデータ合成
  - サンプリングで 1個の質問から2個の異なる解答を生成
  - LLM-as-a-Judge でどちらの解答が優れているか判定
  - 位置バイアスの軽減のために解答の順番をスワップして再度判定
  - 一貫性のある判決を下したデータをプリファレンスデータとして採用


## 参考文献

- Lianmin Zheng et al., "Judging LLM-as-a-Judge with MT-Bench and Chatbot Arena", NeurIPS 2023, 2023. https://proceedings.neurips.cc/paper_files/paper/2023/hash/91f18a1287b398d378ef22505bf41832-Abstract-Datasets_and_Benchmarks.html
- Jiawei Gu et al., "A Survey on LLM-as-a-Judge", arXiv preprint arXiv:2411.15594v3, 2024. https://arxiv.org/abs/2411.15594
- Rafael Rafailov et al., "Direct Preference Optimization: Your Language Model is Secretly a Reward Model", NeurIPS 2023, 2023. https://papers.nips.cc/paper_files/paper/2023/hash/a85b405ed65c6477a4fe8302b5e06ce7-Abstract-Conference.html
- Hanze Dong et al., "RLHF Workflow: From Reward Modeling to Online RLHF", arXiv preprint arXiv:2405.07863v3, 2024. https://arxiv.org/abs/2405.07863
- Kan Hatakeyama, "大規模言語モデルを開発するにあたっての事前・事後学習の戦略メモー特に合成データについてー", https://zenn.dev/matsuolab/articles/34036f017fae9e, アクセス日: 2025-02-01
- Someya, "Tanuki-8B, 8x8B - 事後学習の軌跡", https://zenn.dev/matsuolab/articles/62c75674190a41, アクセス日: 2025-02-01
- Arata, "TanukiモデルのAWQ、GPTQ、GGUF量子化について", https://zenn.dev/matsuolab/articles/2857bf0feeeb5d, アクセス日: 2025-02-01
- Arata, "Tanuki-8BにMagpieを適用して日本語の合成対話データセットを作成する", https://zenn.dev/aratako_lm/articles/a5ae43fb2bfbb3, アクセス日: 2025-02-01
- knishimae, "Tanuki-8B, 8x8B - Direct Preference Optimization (DPO)実行(11/24日勉強会公開用)", https://zenn.dev/matsuolab/articles/62d99af24ff89a, アクセス日: 2025-02-01
- Mitsuhashi, "プロンプト進化を用いた日本語選好データセットの構築", https://zenn.dev/matsuolab/articles/10a1aa9d43e4fe, アクセス日: 2025-02-01
- Mさん, "Tanuki8Bに対するMT-Benchを用いた評価を体験してみる", https://zenn.dev/matsuolab/articles/2aafa8a7ba7482, アクセス日: 2025-02-01
- Kan Hatakeyama, "大規模言語モデル Tanuki-8x8B の紹介と開発経緯など", 9/10 松尾研 LLM 開発プロジェクト "Tanuki-8x8B" 開発成果報告会 Vol.1, https://www.docswell.com/s/matsuo-lab_llm/51R2L4-2024-9-10-Tanuki%E9%96%8B%E7%99%BA%E5%A0%B1%E5%91%8A%E4%BC%9A-vol1, https://www.youtube.com/watch?v=IcpXpX-r6ZY, アクセス日: 2025-01-28
- Susumu Ota, "Persona-Hub による合成データ生成", 9/24 松尾研 LLM 開発プロジェクト "Tanuki-8x8B" 開発成果報告会 Vol. 3, https://www.docswell.com/s/matsuo-lab_llm/ZDNGR4-2024-9-24-Tanuki%E9%96%8B%E7%99%BA%E5%A0%B1%E5%91%8A%E4%BC%9A-vol3, https://www.youtube.com/watch?v=XAdc-OgLeOw, アクセス日: 2025-01-28
- 松尾研LLMコミュニティメンバー, "「小型LlamaモデルのTRLライブラリを用いた事後学習」松尾研 LLM コミュニティ 勉強会シリーズ#3 (2024-11-24)", https://www.youtube.com/watch?v=ezNx6tB8jac, アクセス日: 2025-02-01
- [NEDO 採択プロジェクト] 多様な日本語能力の向上を目指した公開の基盤モデル開発, コードレポジトリ "iterative-dpo",アクセス日: 2025-01-28, https://github.com/matsuolab/nedo_project_code/tree/team_hatakeyama_phase2/team_hatakeyama_phase2/ota/iterative-dpo

以上